In [1]:
from keras.layers import Input, Lambda, Dense, Flatten, Dropout, BatchNormalization, Conv2D
from keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
train_datagen = ImageDataGenerator(rescale=1/255., 
                                   rotation_range=65,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255.)

train_data = train_datagen.flow_from_directory('Data/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_data = test_datagen.flow_from_directory('Data/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 600 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [3]:
# Ajuste de tamaño de imagenes a 224x224
IMAGE_SIZE = [224, 224]

train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

# Se agrega el modelo preentrenado de VGG16
inception = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [4]:
inception.trainable = False

In [5]:
#Otra forma de conseguir el numero de clases que hay
folders = glob('Data/Train/*')

In [6]:
x = Flatten()(inception.output)
x = Dense(200, activation='relu')(x)
x = Dropout(0.7)(x)
x = Dense(100, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(80, activation='relu')(x)
x = Dense(50, activation='sigmoid')(x)
prediction = Dense(len(folders), activation='softmax')(x)

In [7]:
modelo = Model(inputs=inception.input, outputs=prediction)
modelo.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [8]:
modelo.compile( loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [9]:
history = modelo.fit(train_data, validation_data=test_data, epochs=50, batch_size=15)

Epoch 1/50
19/19 [==============================] - 45s 1s/step - loss: 1.1456 - accuracy: 0.3700 - val_loss: 1.0162 - val_accuracy: 0.3333
Epoch 2/50
19/19 [==============================] - 10s 531ms/step - loss: 1.1337 - accuracy: 0.3433 - val_loss: 0.9889 - val_accuracy: 0.3400
Epoch 3/50
19/19 [==============================] - 10s 539ms/step - loss: 1.1285 - accuracy: 0.3850 - val_loss: 0.9180 - val_accuracy: 0.6567
Epoch 4/50
19/19 [==============================] - 10s 549ms/step - loss: 1.0974 - accuracy: 0.4033 - val_loss: 0.8852 - val_accuracy: 0.7783
Epoch 5/50
19/19 [==============================] - 10s 544ms/step - loss: 1.1108 - accuracy: 0.3883 - val_loss: 0.8411 - val_accuracy: 0.6583
Epoch 6/50
19/19 [==============================] - 11s 555ms/step - loss: 1.0705 - accuracy: 0.4050 - val_loss: 0.9398 - val_accuracy: 0.6800
Epoch 7/50
19/19 [==============================] - 10s 552ms/step - loss: 1.0387 - accuracy: 0.4617 - val_loss: 0.6789 - val_accuracy: 0.9767
Ep

In [10]:
modelo.save('Reconocimiento_Facial_Modelo.h5')